<a href="https://colab.research.google.com/github/JanEggers-hr/youtube-scraper/blob/main/youtube_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Youtube-Scraper

Holt die Videos des übergebenen Youtube-Kanals, isoliert das Audio, und verschriftlicht sie mit einem Speech-to-Text-Service. 

Die einzelnen Schritte:
- Mit youtube_dl nacheinander Videos holen, zu MP3 wandeln
- Die dabei gesammelten wichtigsten Metadaten (Upload-Datum, View-Count) in einer Excel-Datei ablegen
- Mit der OpenAI-Library "Whisper" in Text konvertieren

Dateien liegen danach alle in einem Ordner "output" und müssen von da exportiert werden, weil sie sonst am Ende der Colab-Laufzeit gelöscht werden. Wär doch schade drum. 

## N00bwarnung

Das Skript ist nicht sehr elegant, vor allem aber ist es desaströs langsam. Die youtube_dl-Library lässt sich, soweit ich das erforschen konnte, nicht asynchron ausführen, was schade ist - man könnte ja durchaus mehrere Videos gleichzeitig herunterladen. Vor allem aber könnte man sie Whisper schon mal zum Transkribieren geben. 

So immerhin ist alles schön einfach - wenn auch nicht schnell. 

## GPU einschalten!

Die Whisper-Library profitiert sehr davon, wenn man im Menü unter "Laufzeit/Laufzeittyp ändern" die GPU aktiviert. (Auch wenn Google zunächst meckert, weil das Herunterladen der Videos ohne GPU-Nutzung abgeht.)

Theoretisch könnte man vermutlich auch eine ffmpeg-Variante einbinden, die die GPU nutzt, dann geht der YT-Download schneller... aber: siehe oben. 




In [ ]:
# Vorbereitung: youtube_dl installieren
!pip install youtube_dl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.9 MB 4.6 MB/s 


In [ ]:
# Für die Datensicherung: Drive verbinden
import os
from google.colab import drive
drive.mount('/content/gdrive')

# Ausgabeverzeichnis (/gdrive/MyDrive/youtube-scraper/output) anlegen: 
if not os.path.exists("/content/gdrive/MyDrive/youtube-scraper"):
    os.mkdir("/content/gdrive/MyDrive/youtube-scraper")
if not os.path.exists("/content/gdrive/MyDrive/youtube-scraper/output"):
    os.mkdir("/content/gdrive/MyDrive/youtube-scraper/output")

path = "/content/gdrive/MyDrive/youtube-scraper/output"
os.chdir(path)

Mounted at /content/gdrive


Hier den Kanal eintragen, der gescraped werden soll. 

In [ ]:
channel_url = "https://www.youtube.com/@JanEggers"

Mit der Library youtube_dl wird eine Liste der Videos mit Metadaten als Tabelle erstellt. Das wird vom Download getrennt, um Abbrüche auffangen zu können - manchmal scheitert youtube_dl an einem "403 FORBIDDEN" der Plattform. 

Also: in der ersten Runde nur Daten sammeln - und als XLSX exportieren. 

In [ ]:
from __future__ import unicode_literals
import youtube_dl
import pandas as pd

# Die Optionen, um neben den Metadaten gleich alle MP3-Dateien herunterzuladen: 
ydl_opts = { }

# Erste Aufgabe: Hole Metadaten für einen Channel, keine Downloads
with youtube_dl.YoutubeDL(ydl_opts) as ydl:
    metadata = ydl.extract_info(channel_url, download=False) 

# Die Daten sind ein bisserl verschachtelt: Die URLs der Videos sind als Dictionary in einer Liste von Dictionaries oder so. Das hier funktioniert
videos_df = pd.DataFrame(metadata['entries'][0]['entries'], columns=["id","upload_date","description","duration","view_count","average_rating",
                                                                     "age_limit","categories","tags"])

# Liste exportieren
videos_df.to_excel("video_liste.xlsx")
videos_df.head(5)

[youtube:tab] @fragunsdoch4752: Downloading webpage
[download] Downloading playlist: Frag uns doch! - Home
[youtube:tab] playlist Frag uns doch! - Home: Downloading 1 videos
[download] Downloading video 1 of 1
[youtube:tab] @fragunsdoch4752: Downloading webpage
[download] Downloading playlist: Frag uns doch! - Videos
[youtube:tab] Downloading page 1
[youtube:tab] Downloading page 2
[youtube:tab] Downloading page 3
[youtube:tab] playlist Frag uns doch! - Videos: Downloading 93 videos
[download] Downloading video 1 of 93
[youtube] KBOUxQy9U7g: Downloading webpage
[download] Downloading video 2 of 93
[youtube] a6zN3gBAXNs: Downloading webpage
[download] Downloading video 3 of 93
[youtube] 2eFmqTMGXJ8: Downloading webpage
[download] Downloading video 4 of 93
[youtube] twCXC-Xy4io: Downloading webpage
[download] Downloading video 5 of 93
[youtube] ysBQMhpH9F4: Downloading webpage
[download] Downloading video 6 of 93
[youtube] NTAsIRjtP0Q: Downloading webpage
[download] Downloading video 7 o

,id,upload_date,description,duration,view_count,average_rating,age_limit,categories,tags
0,KBOUxQy9U7g,20201124,Siehe Video 157,699,43658,None,0,[News & Politics],[]
1,a6zN3gBAXNs,20201123,"Wir sprechen über Dominion, K. Biden, Sidney P...",699,60262,None,0,[News & Politics],[]
2,2eFmqTMGXJ8,20201118,"Wir sprechen über Dominion-Server, MSM, Start ...",1503,52846,None,0,[News & Politics],[]
3,twCXC-Xy4io,20201105,Wir sprechen über die Hintergründe der aktuell...,244,31869,None,0,[News & Politics],[]
4,ysBQMhpH9F4,20201105,Wir sprechen über die Wahlen in den USA und de...,415,39819,None,0,[News & Politics],[]


## Videos herunterladen

*ACHTUNG*: Dieser Schritt dauert eine Weile - und bricht gern mal mit einem Fehler ab, weil Youtube dem youtube-dl-Skript gerne mal ein "Darfste nicht!" in den Weg wirft. **Falls diese Code-Zelle mit einem Fehler abbricht, einfach nochmal ausführen!**

Dummerweise verliert das Colab-Notebook nach einer Zeit die virtuelle Maschine, und man muss alles nochmal von vorn starten - deshalb schaut der Code, welche Videos schon heruntergeladen sind, und macht da weiter, wo es zuletzt aufgehört hat. 

In [ ]:
# Die Optionen, um neben den Metadaten gleich alle MP3-Dateien herunterzuladen: 
ydl_opts = {'format': 'bestaudio/best',
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'mp3',
        'preferredquality': '128',
    }],
    'outtmpl': '%(id)s.%(ext)s', # Formatiere Dateiname: id.mp3
    }

# Leere Liste anlegen
new_urls = []
# Videos, für die es noch kein mp3 gibt, in die Liste
for id in videos_df["id"]:
    f = path + "/" + id + ".mp3"
    if not os.path.exists(f):
        new_urls.append(id)

if len(new_urls) > 0:
    print("Noch ",len(new_urls)," Videos herunterladen...")
    # Die Liste an den Downloader verfüttern.
    # Videos werden nach dem Download in MP3 gewandelt, das
    # bestimmen die Parameter. 
    with youtube_dl.YoutubeDL(ydl_opts) as ydl:
                ydl.download(new_urls)

print("Alle Videos des Kanals heruntergeladen!")

[youtube] ECmhS-sotDY: Downloading webpage
[download] Destination: ECmhS-sotDY.m4a
[download] 100% of 45.58MiB in 11:25
[ffmpeg] Correcting container in "ECmhS-sotDY.m4a"
[ffmpeg] Destination: ECmhS-sotDY.mp3
Deleting original file ECmhS-sotDY.m4a (pass -k to keep)
[youtube] T3jVBNCT5vA: Downloading webpage
[youtube] T3jVBNCT5vA: Downloading MPD manifest
[download] Destination: T3jVBNCT5vA.m4a
[download] 100% of 16.50MiB in 06:37
[ffmpeg] Correcting container in "T3jVBNCT5vA.m4a"
[ffmpeg] Destination: T3jVBNCT5vA.mp3
Deleting original file T3jVBNCT5vA.m4a (pass -k to keep)
[youtube] C9WaF241u9Y: Downloading webpage
[download] Destination: C9WaF241u9Y.m4a
[download] 100% of 34.29MiB in 11:25
[ffmpeg] Correcting container in "C9WaF241u9Y.m4a"
[ffmpeg] Destination: C9WaF241u9Y.mp3
Deleting original file C9WaF241u9Y.m4a (pass -k to keep)


Die Audios liegen alle als MP3 im Ordner output - mit den Dateinamen (id).mp3. Jetzt alle an den STT-Konverter schicken. 

Wir versuchen hier an dieser Stelle mal, OpenAIs "Whisper" einzusetzen. 

Quelle: https://github.com/openai/whisper

In [ ]:
!pip install git+https://github.com/openai/whisper.git 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-exyt4uhf
  Running command git clone -q https://github.com/openai/whisper.git /tmp/pip-req-build-exyt4uhf
     |████████████████████████████████| 5.8 MB 4.7 MB/s 
     |████████████████████████████████| 7.6 MB 60.4 MB/s 
     |████████████████████████████████| 182 kB 74.2 MB/s 
  Created wheel for whisper: filename=whisper-1.0-py3-none-any.whl size=1175324 sha256=870392510d33e4b214bb8c6285b7e47775964d28d04c6b86d69d3982e1b44c00
  Stored in directory: /tmp/pip-ephem-wheel-cache-iki46udc/wheels/a7/70/18/b7693c07b1d18b3dafb328f5d0496aa0d41a9c09ef332fd8e6
Successfully built whisper


Wenn die Installation der Library von Github geklappt hat, ist die eigentliche Transkription ziemlich simpel: 

In [ ]:
import whisper
model = whisper.load_model("medium")

# Wie oben: Liste von allen
new_urls = []
for id in videos_df["id"]:
    f = path + "/" + id + "_transcribe.txt"
    if not os.path.exists(f):
        new_urls.append(id)

print(len(new_urls)," MP3-Dateien zu verschriftlichen.")

for id in new_urls:
    mp3_fname = path + "/" + id + ".mp3"
    txt_fname = path + "/" + id + "_transcribe.txt"

    result = model.transcribe(mp3_fname)
    # Ergebnis der Umwandlung als Textdatei ausgeben
    with open(txt_fname, 'w') as f:
      f.write(result["text"])
    print(txt_fname," erzeugt")

print("Fertig - ",len(new_urls)," Dateien konvertiert.")